# 実験を終了する

現在の実験パッケージをGIN-forkに保存して、この実験実行環境を停止・削除します。

実験フロートップページに戻る場合は[こちら](../experiment.ipynb)。新規タブで開きます。  

## 0. 研究リポジトリ名・実験パッケージ名を確認する  
以下のセルを実行すると、この実験実行環境で操作する実験パッケージの名前と、実験パッケージの存在する研究リポジトリ名を確認できます。  


In [ ]:
import os
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils

%store -r
if 'EXPERIMENT_TITLE' not in locals().keys() : EXPERIMENT_TITLE = '-'
utils.show_name('blue', EXPERIMENT_TITLE)

## 1. 実行環境構成を記録する

実行環境構成情報を`environment.yml`と`requirements.txt`に書き出します。

以下のセルを実行して、condaの環境名を選択すると、その環境の構成情報が自動で作成されます。

操作者により、condaの環境を作成してない場合は`base`を選択してください。

もし、誤った環境名を選択した場合は、セルを再実行してください。

In [ ]:
%store -r EXPERIMENT_TITLE

from ipywidgets import Dropdown, Button, Layout
from IPython.display import clear_output

env_info = !conda env list
env_list = []
for env in env_info:
    if '/opt/conda' in env:
        env = env.split(' ')
        env = env[0]
        env_list.append(env)

def on_click_callback(clicked_button: Button) -> None:
    global env_name
    env_name=dropdown.value
    clear_output()
    print("実行環境構成情報ファイルを作成しました。：", env_name)
    !conda env export -n $activate_env > ~/experiments/$EXPERIMENT_TITLE/environment.yml
    !pip freeze > ~/experiments/$EXPERIMENT_TITLE/requirements.txt

dropdown = Dropdown(
    options=env_list,
    description='仮想環境名:',
    disabled=False,
    style= {'description_width': 'initial'}
)

button = Button(description='入力完了')
button.on_click(on_click_callback)
print("仮想環境名の選択後、入力完了ボタンを押下してください。")
display(dropdown, button)

## 2. 実験パッケージをGIN-forkに同期する

実験パッケージをGIN-forkに同期します。  
以下のセルを実行してください。

In [ ]:
from IPython.display import display, Javascript
display(Javascript('IPython.notebook.save_checkpoint();'))

In [ ]:
import os
import glob

%store -r EXPERIMENT_TITLE
experiment_path = '/home/jovyan/experiments/' + EXPERIMENT_TITLE
os.chdir(experiment_path)

#**************************************************#
#* Generate a list of folder paths to be managed by Git-annex. #
#**************************************************#
dirlist=[]
filelist=[]
annexed_save_path=[]

# Recursively search under the experimental package to obtain a list of absolute directory paths.
for root, dirs, files in os.walk(top=experiment_path):
    for dir in dirs:
        dirPath = os.path.join(root, dir)
        dirlist.append( dirPath )   


# Add directory paths containing the string "output_data" that are not included under input_data to annexed_save_path.
output_data_path = [ s for s in dirlist if 'output_data' in s ]
for output_data in output_data_path:  
    if  "input_data" not in output_data:
        annexed_save_path.append( output_data )

# Add the input_data directory to annexed_save_path.
annexed_save_path.append( experiment_path + '/input_data'  )

# Generate a list of file paths to which metadata is to be assigned.
gitannex_files = []
for path in annexed_save_path:
    gitannex_files += [p for p in glob.glob(path+'/**', recursive=True)
             if os.path.isfile(p)]

#********************************************************#
#* Generate a list of directory paths and file paths to be managed by Git. #
#********************************************************#
# Obtain a list of directories and files directly under the experimental package.
files = os.listdir()

# Delete Git-annex managed directories (input_data and output_data) from the retrieved list.
dirs = [f for f in files if os.path.isdir(f)]

for dirname in dirs:
    if dirname == 'input_data' :
        dirs.remove('input_data')

    if dirname == 'output_data' :
        dirs.remove('output_data')
    
# Obtain files directly under the experimental package.
files = [f for f in files if os.path.isfile(f)]

# Generate a list of folder paths and file paths to be managed by Git.
files.extend(dirs)
save_path = []
for file in files:
    save_path.append(experiment_path + '/' + file)
save_path.append('/home/jovyan/pipeline.json')

以下を実行して、`リポジトリ側の変更と競合しました。競合を解決してください。`と表示された場合は、[こちらのFAQ](https://dg.nii.ac.jp/G-Node/Info/wiki/%E3%83%AA%E3%82%B5%E3%83%BC%E3%83%81%E3%83%95%E3%83%AD%E3%83%BC%E6%A9%9F%E8%83%BD#1-1%E5%90%8C%E6%9C%9F%E5%87%A6%E7%90%86%E3%82%92%E5%AE%9F%E8%A1%8C%E3%81%99%E3%82%8B%E3%81%A8%E3%80%81%E3%83%AA%E3%83%9D%E3%82%B8%E3%83%88%E3%83%AA%E5%81%B4%E3%81%AE%E5%A4%89%E6%9B%B4%E3%81%A8%E7%AB%B6%E5%90%88%E3%81%97%E3%81%BE%E3%81%97%E3%81%9F%E3%80%82%E7%AB%B6%E5%90%88%E3%82%92%E8%A7%A3%E6%B1%BA%E3%81%97%E3%81%A6%E3%81%8F%E3%81%A0%E3%81%95%E3%81%84%E3%80%82%E3%81%A8%E8%A1%A8%E7%A4%BA%E3%81%95%E3%82%8C%E3%82%8B)を参考に競合を解決してください。

In [ ]:
import os
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils
os.chdir(os.environ['HOME'])

utils.syncs_with_repo(git_path=save_path, gitannex_path=annexed_save_path, gitannex_files=gitannex_files, message=EXPERIMENT_TITLE + '_実験終了')

## 3. この実験実行環境を停止・削除する

### 3-1. この実行環境を確認する

以下のセルを実行して実行環境のサーバー名を確認して下さい。  

In [ ]:
import os
from IPython.display import display, HTML 
server_name_text = "<h2>" + os.environ["JUPYTERHUB_SERVICE_PREFIX"].split('/')[3] + "</h2>" 
display(HTML(server_name_text))

### 3-2. コントロールパネルへ遷移し、実行環境を停止・削除する

以下のセルを実行し、表示されたリンクをクリックしてコントロールパネルへ遷移して、`3-1`で確認したサーバーを`stop`、`delete`ボタンをクリックして停止・削除してください。  
※ `delete`ボタンは、以下の図のように`stop`ボタンをクリックした後に表示されます。  
![コンテナ削除キャプチャ](./images/コンテナ削除キャプチャ.png)

In [ ]:
import os
import re
from IPython.display import display, Javascript, clear_output

pattern = "https?://[\w/:%#\$&\?\(\)~\.=\+\-]+"

print('当画面のURLをブラウザからコピーし、フォームに入力してください。\n入力完了後、Enterキーを押下してください。')
url = input()

if re.match(pattern, url):
    print(url.find("/user/"))
    url = url[:url.find("/user/")]  + '/hub/home'
    clear_output()
    print('コントロールパネルへのURLはこちらです。')
    print( url )    
else:  
    clear_output()
    print("入力されたURL形式が不正なため、もう一度実行してください。" )
    print( url )

display(Javascript('IPython.notebook.save_checkpoint();'))